In [66]:
import numpy as np
from scipy import stats as st

In [67]:
theta = 10
n = 100
sample = theta * (np.random.rand(n) + 1)
sample

array([11.41633711, 18.35713299, 19.75357581, 14.93995125, 16.50801982,
       16.21497538, 11.90137082, 13.85095356, 14.40366388, 16.40366954,
       12.02131216, 13.55641257, 17.19844602, 12.58231501, 19.71928129,
       19.58628029, 16.09081235, 14.52216543, 16.25127111, 10.6062175 ,
       19.94462011, 19.22893094, 10.560234  , 18.1478285 , 12.26480411,
       11.51606363, 15.36216639, 15.04267696, 16.88313621, 15.38500294,
       17.6009016 , 11.72321043, 11.48591608, 16.84934337, 13.2948652 ,
       14.1609732 , 18.32802881, 13.89475797, 11.57742632, 13.91944395,
       15.3733781 , 17.08385407, 13.02093433, 11.03361109, 16.48646985,
       13.07568494, 19.06295157, 12.65575563, 18.00625244, 14.11514089,
       16.88398174, 11.17345622, 10.80327049, 11.29848622, 10.68319302,
       14.72246134, 13.23506558, 14.58811408, 19.54260696, 16.21050524,
       19.75585033, 13.31629088, 17.55795061, 18.20998259, 11.26240339,
       19.90413606, 13.51413941, 13.6766053 , 17.16094601, 11.70

In [68]:
theta_assessment = (len(sample) + 1) / (2 * len(sample) + 1) * np.max(sample)
theta_assessment

np.float64(10.02192353911034)

In [69]:
# f)
exact_ci = (
    float(np.max(sample) / (1 + np.pow(0.975, 1 / n))),
    float(np.max(sample) / (1 + np.pow(0.025, 1 / n)))
)

alpha_1_assessment = np.mean(sample)
sqrt_mu_2_assessment = np.std(sample)

asymptotic_ci = (
    float(-1.96 * np.sqrt(4 / (n * 9)) * sqrt_mu_2_assessment + 2 / 3 * alpha_1_assessment),
    float(1.96 * np.sqrt(4 / (n * 9)) * sqrt_mu_2_assessment + 2 / 3 * alpha_1_assessment)
)

print(
    f"{exact_ci = }",
    f"{asymptotic_ci = }",
    sep="\n\n"
)

exact_ci = (9.973572441392532, 10.156222449691718)

asymptotic_ci = (9.474967595582477, 10.27114555860097)


In [72]:
# g)
from collections.abc import Callable
from typing import NamedTuple


class ConfidenceInterval(NamedTuple):
    low: float
    high: float


def parametric_bootstrap(
    sample: np.ndarray,
    statistic: Callable[[np.ndarray], float],
    prob_model: Callable[[int], np.ndarray],
    n_resamples: int = 10_000,
    confidence_level: float = 0.95,
) -> ConfidenceInterval:
    n = len(sample)
    bootstrap_data = np.ndarray((n_resamples, ), dtype=np.float32)

    for i in range(n_resamples):
        resample = prob_model(n)
        bootstrap_data[i] = statistic(resample)
    
    bootstrap_data.sort()

    return ConfidenceInterval(
        low=bootstrap_data[round((1 - confidence_level) / 2 * n_resamples) - 1],
        high=bootstrap_data[round((1 + confidence_level) / 2 * n_resamples) - 1]
    )


statistic = lambda x: (n + 1) / (2 * n + 1) * np.max(x) - theta_assessment

nonparametric_bootstrap_ci = st.bootstrap(
    (sample, ), 
    statistic, 
    n_resamples=1_000, 
    method='basic',
    confidence_level=0.95
).confidence_interval
nonparametric_bootstrap_ci = (
    float(theta_assessment - nonparametric_bootstrap_ci.high),
    float(theta_assessment - nonparametric_bootstrap_ci.low)
)

parametric_bootstrap_ci = parametric_bootstrap(
    sample,
    statistic,
    lambda n: np.random.uniform(theta_assessment, theta_assessment * 2, n),
    n_resamples=50_000, 
    confidence_level=0.95
)
parametric_bootstrap_ci = (
    float(theta_assessment - parametric_bootstrap_ci.high),
    float(theta_assessment - parametric_bootstrap_ci.low)
)

print(
    f"{nonparametric_bootstrap_ci = }",
    f"{parametric_bootstrap_ci = }",
    sep="\n\n"
)

nonparametric_bootstrap_ci = (9.984728973099804, 10.02192353911034)

parametric_bootstrap_ci = (9.973326348581232, 10.154016283222354)


In [73]:
# h)
print(
    f"{exact_ci = }\nl = {exact_ci[1] - exact_ci[0]}",
    f"{asymptotic_ci = }\nl = {asymptotic_ci[1] - asymptotic_ci[0]}",
    f"{nonparametric_bootstrap_ci = }\nl = {nonparametric_bootstrap_ci[1] - nonparametric_bootstrap_ci[0]}",
    f"{parametric_bootstrap_ci = }\nl = {parametric_bootstrap_ci[1] - parametric_bootstrap_ci[0]}",
    sep="\n\n"
)

exact_ci = (9.973572441392532, 10.156222449691718)
l = 0.18265000829918598

asymptotic_ci = (9.474967595582477, 10.27114555860097)
l = 0.7961779630184935

nonparametric_bootstrap_ci = (9.984728973099804, 10.02192353911034)
l = 0.03719456601053572

parametric_bootstrap_ci = (9.973326348581232, 10.154016283222354)
l = 0.18068993464112282
